<a href="https://colab.research.google.com/github/NeeharikaBhaide/TDS_P1/blob/main/TDS_P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from time import sleep

# Constants
GITHUB_TOKEN =
HEADERS = {"Authorization": f"token {GITHUB_TOKEN}"}
CITY = "Austin"
MIN_FOLLOWERS = 100

# Function to get users based on location and followers count
def get_users():
    users = []
    page = 1
    while True:
        url = f"https://api.github.com/search/users?q=location:{CITY}+followers:>{MIN_FOLLOWERS}&page={page}&per_page=100"
        response = requests.get(url, headers=HEADERS)
        data = response.json()

        if "items" not in data:
            break

        for item in data["items"]:
            user_url = item["url"]
            user_data = requests.get(user_url, headers=HEADERS).json()
            users.append({
                "login": user_data.get("login"),
                "name": user_data.get("name"),
                "company": clean_company(user_data.get("company")),
                "location": user_data.get("location"),
                "email": user_data.get("email", ""),
                "hireable": user_data.get("hireable", ""),
                "bio": user_data.get("bio", ""),
                "public_repos": user_data.get("public_repos"),
                "followers": user_data.get("followers"),
                "following": user_data.get("following"),
                "created_at": user_data.get("created_at"),
            })

        page += 1
        if len(data["items"]) < 100:
            break
        sleep(1)  # Rate limit compliance

    return pd.DataFrame(users)

# Helper function to clean company names
def clean_company(company):
    if company:
        company = company.strip().lstrip('@').upper()
    return company if company else ""

# Function to get repositories for each user
def get_repositories(login):
    repos = []
    page = 1
    while True:
        url = f"https://api.github.com/users/{login}/repos?page={page}&per_page=100"
        response = requests.get(url, headers=HEADERS)
        repo_data = response.json()

        if not repo_data:
            break

        for repo in repo_data:
            repos.append({
                "login": login,
                "full_name": repo.get("full_name"),
                "created_at": repo.get("created_at"),
                "stargazers_count": repo.get("stargazers_count"),
                "watchers_count": repo.get("watchers_count"),
                "language": repo.get("language", ""),
                "has_projects": repo.get("has_projects", False),
                "has_wiki": repo.get("has_wiki", False),
                "license_name": repo["license"]["key"] if repo.get("license") else "",
            })

        page += 1
        if len(repo_data) < 100:
            break
        sleep(1)  # Rate limit compliance

    return pd.DataFrame(repos)

# Main function to generate CSVs
def main():
    # Fetch and save users
    users_df = get_users()
    users_df.to_csv("users.csv", index=False)

    # Fetch and save repositories
    all_repos = pd.DataFrame()
    for login in users_df["login"]:
        user_repos = get_repositories(login)
        all_repos = pd.concat([all_repos, user_repos], ignore_index=True)
        sleep(1)  # Rate limit compliance between users

    all_repos.to_csv("repositories.csv", index=False)

if __name__ == "__main__":
    main()


In [ ]:
# Q-1

# Load the users data
users_df = pd.read_csv("users.csv")

# Sort users by followers in descending order and select the top 5
top_5_users = users_df.sort_values(by="followers", ascending=False).head(5)

# Extract the 'login' column and join them into a comma-separated string
top_5_logins = ", ".join(top_5_users["login"])

print("Top 5 users with the highest followers:", top_5_logins)


Top 5 users with the highest followers: getify, benawad, steveklabnik, cloudflare, jbogard


In [ ]:
# Q-2

# Load the users data
users_df = pd.read_csv("users.csv")

# Convert created_at to datetime format to ensure proper sorting
users_df["created_at"] = pd.to_datetime(users_df["created_at"])

# Sort users by created_at in ascending order and select the top 5
earliest_users = users_df.sort_values(by="created_at", ascending=True).head(5)

# Extract the 'login' column and join them into a comma-separated string
earliest_logins = ",".join(earliest_users["login"])

print("5 earliest registered GitHub users in Austin:", earliest_logins)


5 earliest registered GitHub users in Austin: jnewland,joshknowles,hassox,dan,damon


In [ ]:
# Q-3

# Load the repositories data
repos_df = pd.read_csv("repositories.csv")

# Filter out rows with missing license names
filtered_repos = repos_df[repos_df["license_name"].notna() & (repos_df["license_name"] != "")]

# Count the occurrences of each license and get the top 3 most common licenses
top_licenses = filtered_repos["license_name"].value_counts().head(3)

# Extract the license names and join them into a comma-separated string
top_licenses_names = ",".join(top_licenses.index)

print("3 most popular licenses:", top_licenses_names)


3 most popular licenses: mit,apache-2.0,other


In [ ]:
# Q-4

# Load users data
users_df = pd.read_csv("cleaned_users.csv")  # Assuming this is the cleaned file you created

# Filter non-empty company names and convert to uppercase
users_df['company'] = users_df['company'].str.strip().str.upper()
companies = users_df[users_df['company'] != '']['company']

# Find the most common company
most_common_company = companies.mode().iloc[0]  # The mode gives the most common value

print("Company with the most developers:", most_common_company)


Company with the most developers: GOOGLE


In [ ]:
# Q-5

# Load the repositories data
repos_df = pd.read_csv("repositories.csv")

# Filter out rows with missing language values
filtered_repos = repos_df[repos_df["language"].notna() & (repos_df["language"] != "")]

# Count the occurrences of each language and get the most common one
most_popular_language = filtered_repos["language"].value_counts().idxmax()

print("Most popular programming language:", most_popular_language)


Most popular programming language: JavaScript


In [ ]:
# Q-6

# Load the users and repositories data
users_df = pd.read_csv("users.csv")
repos_df = pd.read_csv("repositories.csv")

# Convert created_at to datetime format
users_df["created_at"] = pd.to_datetime(users_df["created_at"])

# Filter users who joined after 2020
recent_users = users_df[users_df["created_at"] > "2020-01-01"]

# Filter repositories for these users
recent_repos = repos_df[repos_df["login"].isin(recent_users["login"])]

# Filter out rows with missing language values
filtered_repos = recent_repos[recent_repos["language"].notna() & (recent_repos["language"] != "")]

# Count the occurrences of each language
language_counts = filtered_repos["language"].value_counts()

# Get the second most popular language
second_most_popular_language = language_counts.index[1] if len(language_counts) > 1 else None

print("Second most popular programming language among users who joined after 2020:", second_most_popular_language)


Second most popular programming language among users who joined after 2020: HTML


In [ ]:
# Q-7

# Load the repositories data
repos_df = pd.read_csv("repositories.csv")

# Filter out rows with missing language or stars
filtered_repos = repos_df[repos_df["language"].notna() & (repos_df["language"] != "")]

# Group by language and calculate the average number of stars
avg_stars_per_language = filtered_repos.groupby("language")["stargazers_count"].mean()

# Get the language with the highest average number of stars
highest_avg_stars_language = avg_stars_per_language.idxmax()
highest_avg_stars_value = avg_stars_per_language.max()

print(f"Language with the highest average number of stars per repository: {highest_avg_stars_language} ({highest_avg_stars_value:.2f} stars)")


Language with the highest average number of stars per repository: Fennel (2443.00 stars)


In [ ]:
# Q-8

# Load the users data
users_df = pd.read_csv("users.csv")

# Calculate leader_strength
users_df['leader_strength'] = users_df['followers'] / (1 + users_df['following'])

# Sort users by leader_strength in descending order and select the top 5
top_5_leaders = users_df.sort_values(by='leader_strength', ascending=False).head(5)

# Extract the 'login' column and join them into a comma-separated string
top_5_logins = ",".join(top_5_leaders['login'])

print("Top 5 users in terms of leader_strength:", top_5_logins)


Top 5 users in terms of leader_strength: getify,cloudflare,benawad,oracle,ContinuumIO


In [ ]:
# Q-9

# Load the users data
users_df = pd.read_csv("users.csv")

# Calculate the correlation between followers and public_repos
correlation = users_df['followers'].corr(users_df['public_repos'])

# Print the correlation rounded to 3 decimal places
print("Correlation between followers and public repositories:", round(correlation, 3))


Correlation between followers and public repositories: 0.15


In [ ]:
# Q-10
import statsmodels.api as sm

# Load the users data
users_df = pd.read_csv("users.csv")

# Define the independent variable (public_repos) and dependent variable (followers)
X = users_df['public_repos']
y = users_df['followers']

# Add a constant to the independent variable for the intercept
X = sm.add_constant(X)

# Fit the regression model
model = sm.OLS(y, X).fit()

# Get the slope (coefficient) for public_repos
slope = model.params['public_repos']

# Print the slope rounded to 3 decimal places
print("Regression slope of followers on public repositories:", round(slope, 3))


Regression slope of followers on public repositories: 4.088


In [ ]:
# Q-11

import pandas as pd

# Load users and repositories data
users_df = pd.read_csv("users.csv")
repositories_df = pd.read_csv("repositories.csv")

# Function to convert boolean and null values
def clean_dataframe(df):
    # Replace boolean values
    df.replace({True: 'true', False: 'false'}, inplace=True)

    # Replace NaN values with empty strings
    df.fillna('', inplace=True)

    return df

# Clean both DataFrames
cleaned_users_df = clean_dataframe(users_df)
cleaned_repositories_df = clean_dataframe(repositories_df)

# Save the cleaned DataFrames to new CSV files
cleaned_users_df.to_csv("cleaned_users.csv", index=False)
cleaned_repositories_df.to_csv("cleaned_repositories.csv", index=False)

print("Boolean values converted and nulls replaced in new files.")


Boolean values converted and nulls replaced in new files.


In [ ]:
import pandas as pd

# Load users and repositories data
users_df = pd.read_csv("users.csv")
repositories_df = pd.read_csv("repositories.csv")

# Function to convert boolean and null values
def clean_dataframe(df):
    # Replace boolean values
    df.replace({True: 'true', False: 'false'}, inplace=True)

    # Replace NaN values with empty strings
    df.fillna('', inplace=True)

    return df

# Clean both DataFrames
cleaned_users_df = clean_dataframe(users_df)
cleaned_repositories_df = clean_dataframe(repositories_df)

# Save the cleaned DataFrames back to CSV
cleaned_users_df.to_csv("users.csv", index=False)
cleaned_repositories_df.to_csv("repositories.csv", index=False)

print("Boolean values converted and nulls replaced in both files.")


Boolean values converted and nulls replaced in both files.


In [ ]:
users_df = pd.read_csv("users.csv")
repositories_df = pd.read_csv("repositories.csv")

print(users_df)

            login                    name                       company  \
0          getify            Kyle Simpson              GETIFY SOLUTIONS   
1         benawad                Ben Awad                       VOIDPET   
2    steveklabnik           Steve Klabnik                 OXIDECOMPUTER   
3      cloudflare              Cloudflare                           NaN   
4         jbogard            Jimmy Bogard  JIMMY BOGARD CONSULTING, LLC   
..            ...                     ...                           ...   
465     dbredvick           Drew Bredvick                        VERCEL   
466  mutualmobile           Mutual Mobile                           NaN   
467        mpakus  Ibragimov "MpaK" Renat                        AÖMEGA   
468        AMD-IT     AMD IT Organization                           NaN   
469  matt-landers            Matt Landers                        GOOGLE   

                                             location  \
0                                         

In [ ]:
#Q-11

# import pandas as pd

# # Load the repositories data
# repos_df = pd.read_csv("repositories.csv")

# # Calculate the correlation between has_projects and has_wiki
# # Convert boolean columns to integers (True = 1, False = 0) for correlation calculation

# repos_df['has_projects'] = repos_df['has_projects'].replace({'TRUE': True, 'FALSE': False}).astype(int)
# repos_df['has_wiki'] = repos_df['has_wiki'].replace({'TRUE': True, 'FALSE': False}).astype(int)

# # repos_df['has_projects'] = repos_df['has_projects'].astype(int)
# # repos_df['has_wiki'] = repos_df['has_wiki'].astype(int)

# correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])

# # Print the correlation rounded to 3 decimal places
# print("Correlation between projects and wikis enabled:", round(correlation, 3))


import pandas as pd

# Load the repositories data
repos_df = pd.read_csv("repositories.csv")

# Ensure all values in 'has_projects' and 'has_wiki' are strings, then standardize
# repos_df['has_projects'] = repos_df['has_projects'].astype(str).str.lower().replace({'true': True, 'false': False}).astype(int)
# repos_df['has_wiki'] = repos_df['has_wiki'].astype(str).str.lower().replace({'true': True, 'false': False}).astype(int)

repos_df['has_projects'] = repos_df['has_projects'].astype(bool)
repos_df['has_wiki'] = repos_df['has_wiki'].astype(bool)


# Calculate the correlation between has_projects and has_wiki
correlation = repos_df['has_projects'].corr(repos_df['has_wiki'])

# Print the correlation rounded to 3 decimal places
print("Correlation between projects and wikis enabled:", round(correlation, 3))




Correlation between projects and wikis enabled: 0.273


In [ ]:
# Q-12

users_df = pd.read_csv("users.csv")

# Step 1: Replace NaN and empty strings with 0 in the 'following' column
users_df['hireable'] = users_df['hireable'].fillna(0)  # Replace NaN with 0
users_df['hireable'] = users_df['hireable'].replace('', 0)  # Replace empty strings with 0

# Step 2: Convert to boolean: If following > 0, it will be True; otherwise, False
users_df['hireable'] = users_df['hireable'].astype(bool)


In [ ]:
users_df = pd.read_csv("users.csv")

# Step 1: Convert string representations to proper boolean
users_df['hireable'] = users_df['hireable'].replace({'TRUE': True, 'FALSE': False,
                                                      })

print(users_df)

            login                    name                       company  \
0          getify            Kyle Simpson              GETIFY SOLUTIONS   
1         benawad                Ben Awad                       VOIDPET   
2    steveklabnik           Steve Klabnik                 OXIDECOMPUTER   
3      cloudflare              Cloudflare                           NaN   
4         jbogard            Jimmy Bogard  JIMMY BOGARD CONSULTING, LLC   
..            ...                     ...                           ...   
465     dbredvick           Drew Bredvick                        VERCEL   
466  mutualmobile           Mutual Mobile                           NaN   
467        mpakus  Ibragimov "MpaK" Renat                        AÖMEGA   
468        AMD-IT     AMD IT Organization                           NaN   
469  matt-landers            Matt Landers                        GOOGLE   

                                             location  \
0                                         

In [ ]:
#Q-12

# import pandas as pd

# # Load the users data
# users_df = pd.read_csv("users.csv")

# # Calculate the average following for hireable users
# hireable_avg_following = users_df[users_df['hireable'] == True]['following'].mean()

# # Calculate the average following for non-hireable users
# non_hireable_avg_following = users_df[users_df['hireable'] == False]['following'].mean()

# # Calculate the difference
# average_difference = hireable_avg_following - non_hireable_avg_following

# # Print the average difference rounded to 3 decimal places
# print("Average following per hireable user minus average following for non-hireable users:", round(average_difference, 3))


import pandas as pd

# Load the users data
users_df = pd.read_csv("users.csv")

# Convert 'hireable' column to a boolean (if it contains strings 'TRUE'/'FALSE')
# Replace NaN values with False in 'hireable' column and convert strings to boolean
users_df['hireable'] = users_df['hireable'].replace({'TRUE': True, 'FALSE': False, '': False, ' ': False}).fillna(False)

# users_df['hireable'] = users_df['hireable'].astype(str).str.lower().replace({'': False, 'TRUE': True, 'FALSE': False})
# print(users_df['hireable'])

# Calculate average 'following' for hireable and non-hireable users

tot = users_df['hireable'].count()
print(tot)

hireable_avg_following = len(users_df[users_df['hireable'] == True]['following']) / tot
non_hireable_avg_following = len(users_df[users_df['hireable'] == False]['following']) / tot

# Calculate the difference and round to 3 decimal places
average_difference = round(hireable_avg_following - non_hireable_avg_following, 3)

# Print the result
print("Average of following per hireable user minus average following for non-hireable users:", average_difference)


470
Average of following per hireable user minus average following for non-hireable users: -0.574


<ipython-input-35-4d8f7306c5ef>:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  users_df['hireable'] = users_df['hireable'].replace({'TRUE': True, 'FALSE': False, '': False, ' ': False}).fillna(False)


In [ ]:
# Q- 13

import pandas as pd
from sklearn.linear_model import LinearRegression

# Load the users data
users_df = pd.read_csv("users.csv")

# Step 1: Filter out users without bios
users_with_bios = users_df[users_df['bio'].notna()]

# Step 2: Calculate word count in each bio using .loc to avoid the SettingWithCopyWarning
users_with_bios.loc[:, 'bio_word_count'] = users_with_bios['bio'].str.split().apply(len)

# Step 3: Prepare data for regression analysis
X = users_with_bios['bio_word_count'].values.reshape(-1, 1)
y = users_with_bios['followers'].values

# Step 4: Perform linear regression
model = LinearRegression()
model.fit(X, y)

# Step 5: Get the slope of the regression line (coefficient)
slope = model.coef_[0]

# Print the slope rounded to 3 decimal places
print("Regression slope of followers on bio word count:", round(slope, 3))



Regression slope of followers on bio word count: 9.397


<ipython-input-3-0b401f94093f>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_bios.loc[:, 'bio_word_count'] = users_with_bios['bio'].str.split().apply(len)


In [ ]:
import pandas as pd

# Load the users data
users_df = pd.read_csv("users.csv")

# Step 1: Filter out users without bios
users_with_bios = users_df.dropna(subset=['bio'])

# Step 2: Calculate the word count for each bio
users_with_bios['bio_word_count'] = users_with_bios['bio'].str.split().apply(len)

# Step 3: Calculate the correlation between bio word count and followers
correlation = users_with_bios['bio_word_count'].corr(users_with_bios['followers'])

# Print the correlation rounded to 3 decimal places
print("Correlation between bio word count and followers:", round(correlation, 3))



Correlation between bio word count and followers: 0.02


<ipython-input-38-cf6ff5611331>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_bios['bio_word_count'] = users_with_bios['bio'].str.split().apply(len)


In [ ]:
import pandas as pd
from scipy import stats

# Load the users data
users_df = pd.read_csv("users.csv")

# Step 1: Filter out users without bios
users_with_bios = users_df.dropna(subset=['bio'])

# Step 2: Calculate the word count for each bio
users_with_bios['bio_word_count'] = users_with_bios['bio'].str.split().apply(len)

# Step 3: Perform linear regression on followers vs bio word count
slope, intercept, r_value, p_value, std_err = stats.linregress(users_with_bios['bio_word_count'], users_with_bios['followers'])

# Print the slope rounded to 3 decimal places
print("Regression slope of followers on bio word count:", round(slope, 3))


Regression slope of followers on bio word count: 9.397


<ipython-input-39-efc754269cc9>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users_with_bios['bio_word_count'] = users_with_bios['bio'].str.split().apply(len)


In [ ]:
# Q-14

import pandas as pd

# Load users and repositories data
users_df = pd.read_csv("users.csv")
repos_df = pd.read_csv("repositories.csv")

# Convert 'created_at' to datetime
repos_df['created_at'] = pd.to_datetime(repos_df['created_at'], utc=True)

# Determine if the repository was created on a weekend (Saturday = 5, Sunday = 6)
repos_df['is_weekend'] = repos_df['created_at'].dt.weekday >= 5

# Filter for weekend repositories only
weekend_repos = repos_df[repos_df['is_weekend']]

# Count weekend repositories per user
weekend_repos_count = weekend_repos['login'].value_counts().head(5)

# Get the top 5 users' login in comma-separated format
top_5_users = ",".join(weekend_repos_count.index)

# Output the result
print("Top 5 users who created the most repositories on weekends:", top_5_users)


Top 5 users who created the most repositories on weekends: FellowTraveler,realityexpander,PaulBratslavsky,steveklabnik,OR13


In [ ]:
# Q-15

import pandas as pd

# Load the users data
users_df = pd.read_csv("users.csv")

# Calculate the fraction of hireable users with email addresses
hireable_users = users_df[users_df['hireable'] == True]
fraction_hireable_with_email = hireable_users['email'].notna().mean()

# Calculate the fraction of non-hireable users with email addresses
non_hireable_users = users_df[users_df['hireable'] == False]
fraction_non_hireable_with_email = non_hireable_users['email'].notna().mean()

# Calculate the difference between the two fractions
email_difference = round(fraction_hireable_with_email - fraction_non_hireable_with_email, 3)

# Print the result
print("Difference in fraction of users with email (hireable - non-hireable):", email_difference)


Difference in fraction of users with email (hireable - non-hireable): nan


In [ ]:
# Q-16

# Load users data
users_df = pd.read_csv("users.csv")

# Drop missing names, trim whitespace, and store it in a new DataFrame
users_with_names = users_df[users_df['name'].notna()].copy()
users_with_names.loc[:, 'name'] = users_with_names['name'].str.strip()

# Extract surname as the last word in the name
users_with_names.loc[:, 'surname'] = users_with_names['name'].str.split().str[-1]

# Count occurrences of each surname
surname_counts = users_with_names['surname'].value_counts()

# Find the most common surname(s)
max_count = surname_counts.max()

most_common_surnames = surname_counts[surname_counts == max_count]

print(most_common_surnames)


surname
Moore    3
Smith    3
Name: count, dtype: int64
